In [1]:
# The goal is to deploy the same application into a multinode and a single node vespa cluster with all schemas
# but when deployed to multi-node the two schemas should be deployed to two separate content clusters
# while in single node deploynmetn both schemas live in the same content clusters.
# Also, let's use the single `hosts.xml` file.
# Use preprocess:include directive.
# TODO: have two content clusters with separated schemas

# The single node should be started as a separate docker container.
# Two techniques of variants for container and content clusters
## There is the `include` tag https://docs.vespa.ai/en/reference/applications/services/container.html#include
##

In [27]:
%%bash
docker run \
  --detach \
  --rm \
  --name vespa-demo-dev \
  --hostname vespa-demo-dev \
  --publish 0.0.0.0:8089:8080 \
  --publish 0.0.0.0:19059:19050 \
  --publish 0.0.0.0:19079:19071 \
  vespaengine/vespa:8.625.17

2193fb91ce264717e53bc4745e28c1188967c48484b5d957cfb7b20731b5a5b3


In [33]:
!zip -r application.zip .

updating: hosts.xml (deflated 70%)
updating: services.xml (deflated 56%)
updating: schemas/ (stored 0%)
updating: schemas/doc2.sd (deflated 34%)
updating: schemas/doc1.sd (deflated 33%)
updating: docker-compose.yml (deflated 80%)
updating: session.ipynb (deflated 69%)


In [34]:
%%bash
# Deploy to the vespa-demo-dev container as `instance=dev`
curl -X POST \
        --header "Content-Type:application/zip" \
                 --data-binary @application.zip \
    "http://localhost:19079/application/v2/tenant/default/prepareandactivate?instance=dev"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5087    0   318  100  4769    586   8796 --:--:-- --:--:-- --:--:--  9368- 23038


{"log":[],"message":"Session 4 for tenant 'default' prepared and activated.","session-id":"4","activated":true,"tenant":"default","url":"http://localhost:19071/application/v2/tenant/default/application/default/environment/prod/region/default/instance/dev","configChangeActions":{"restart":[],"refeed":[],"reindex":[]}}

In [35]:
# Check which schemas are deployed to the instance=local-1
!docker exec vespa-demo-dev sh -c "ls /opt/vespa/var/db/vespa/search/cluster.content/n0/documents"

doc1
doc2


In [36]:
# Both indices as expected are in the node

In [26]:
!docker stop vespa-demo-dev

vespa-demo-dev
